<a href="https://colab.research.google.com/github/pchodon/Amegashita/blob/main/main_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import multiprocessing
import time
from csv import reader
from google.colab import files
import pandas as pd
import threading

sensory_1_resp = 1  # default 1 Sec
sensory_2_resp = 1  # default 1 Sec
temporal_resp = 1   # default 1 Sec
thalamus_resp = 1   # default 1 Sec
reset = 0
forever = 1
max_sensory_stream = 5
Max_lapse = 10
mTrue = 1
mFalse = 0
w, h = 10, 10;
hipp_first = False



def sensory_1(conn,msg):

  data = []
  with open('/content/drive/MyDrive/sense_1.csv', 'r') as data:
    df_sensory_1 = pd.read_csv(data)

  i = 0
  while(1):
   msg = df_sensory_1.loc[i]
   print("sensory 1 msg",msg[0])
   conn.send(msg[0])
   time.sleep(sensory_1_resp)
   i = i+1
   if(i==max_sensory_stream):
        break
  return  

def sensory_2(conn,msg):
  data = []
  with open('/content/drive/MyDrive/sense_2.csv', 'r') as data:
    df_sensory_2 = pd.read_csv(data)

  i = 0
  while(1):
   msg = df_sensory_2.loc[i]
   print("sensory 2 msg",msg[0])
   conn.send(msg[0])
   time.sleep(sensory_2_resp)
   i = i+1
   if(i==max_sensory_stream):
        break
  return  

def hippocampus(conn,msg):
  return

def thalamus(conn,xconn,msg):
  maxim = 10
  lapse = 0
  window = False
  source = False
  sense = 0

  while(True):
      time.sleep(temporal_resp)
      sense = conn.recv()
      print('size of sense :',sense.ndim)
      print("thalamus received :",sense)
      if((lapse == sense)):
            print("blank")
            source = False
      if(window==True):
               lapse = lapse + 1
      if(lapse == Max_lapse):
            lapse = 0
            print("leaving 10sec window")
            window = False
# Signal end of window
            msg = 4
            xconn.send(msg)
            continue

            lapse = 0
            window = True
            if(sense == '1'):
                source = 1
            if(sense == '2'):
                source = 2

# Send the signal to hippocampus process        
            msg = sense
            xconn.send(msg)

# As sensory input is received
# Start a timer window  
# Reset the window e experiences no sensory inputs
  return


if __name__ == "__main__":
  print("System Starting......")
  msg = 0

        # creating common pipe name
  parent_conn, child_conn = multiprocessing.Pipe()
  xparent_conn, xchild_conn = multiprocessing.Pipe()

  p1 = multiprocessing.Process(target=sensory_1, args=(parent_conn,msg))
  p2 = multiprocessing.Process(target=sensory_2, args=(parent_conn,msg))
  p3 = multiprocessing.Process(target=thalamus, args=(child_conn,xparent_conn,msg))
  p4 = multiprocessing.Process(target=hippocampus, args=(xchild_conn,msg))

       # running processes
  p1.start()
  p2.start()
  p3.start()
  p4.start()
  time.sleep(10)
  print("Starting new cycle")

p1.join()
p2.join()
p3.join()
p4.join()
time.sleep(2)

print("Starting new episode")
#333444

System Starting......
sensory 1 msg 1
sensory 2 msg 0
size of sense : 0
thalamus received : 0
blank
sensory 2 msg 0
sensory 1 msg 1
size of sense : 0
thalamus received : 1
sensory 2 msg 0
sensory 1 msg 1
size of sense : 0
thalamus received : 0
sensory 2 msg 0
sensory 1 msg 1
blank
sensory 2 msg 0
size of sense : 0
sensory 1 msg 1
thalamus received : 1
size of sense : 0
thalamus received : 1
size of sense : 0
thalamus received : 0
blank
size of sense : 0
thalamus received : 0
blank
size of sense : 0
thalamus received : 1
size of sense : 0
thalamus received : 0
blank
size of sense : 0
thalamus received : 1
Starting new cycle


Process Process-19:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-41a623aea19f>", line 68, in thalamus
    sense = conn.recv()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)


KeyboardInterrupt: ignored

  File "/usr/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
